In [1]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



22/10/05 20:00:31 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/10/05 20:00:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c54fd9a3-7cca-47c9-a496-862203caa5e4;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.1.0 in central
	found com.typesafe#config;1.3.0 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google

Spark NLP version 3.1.0
Apache Spark version: 3.0.2


In [2]:


#download training data
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.train
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/conll2003/eng.testa



In [3]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng.train')
testing_data= CoNLL().readDataset(spark, './eng.testa')



In [4]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("pos"),
        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


22/10/05 20:00:48 WARN TaskSetManager: Stage 0 contains a task of very large size (4185 KiB). The maximum recommended task size is 1000 KiB.


+----------+---+---------+
|     token|pos|ner_label|
+----------+---+---------+
|        EU|NNP|    B-ORG|
|   rejects|VBZ|        O|
|    German| JJ|   B-MISC|
|      call| NN|        O|
|        to| TO|        O|
|   boycott| VB|        O|
|   British| JJ|   B-MISC|
|      lamb| NN|        O|
|         .|  .|        O|
|     Peter|NNP|    B-PER|
| Blackburn|NNP|    I-PER|
|  BRUSSELS|NNP|    B-LOC|
|1996-08-22| CD|        O|
|       The| DT|        O|
|  European|NNP|    B-ORG|
|Commission|NNP|    I-ORG|
|      said|VBD|        O|
|        on| IN|        O|
|  Thursday|NNP|        O|
|        it|PRP|        O|
+----------+---+---------+
only showing top 20 rows



In [5]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [7]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)

embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(1)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ / ]Download done! Loading the resource.
[OK!]


In [8]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[-1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

22/10/05 20:01:17 WARN TaskSetManager: Stage 2 contains a task of very large size (4185 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 20:01:22 WARN TaskSetManager: Stage 3 contains a task of very large size (4185 KiB). The maximum recommended task size is 1000 KiB.
2022-10-05 20:01:25.614803: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 20:01:25.655627: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /j

Training started - total epochs: 1 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 84 - training examples: 11260
Epoch 1/1 started, lr: 0.003, dataset size: 11260
Epoch 1/1 - 23.15s - loss: 916.6593 - batches: 353
Quality on validation dataset (20.0%), validation examples = 2252
time to finish evaluation: 3.31s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 1331	 130	 70	 0.91101986	 0.9500357	 0.9301188
I-ORG	 566	 77	 156	 0.88024884	 0.7839335	 0.829304
I-MISC	 128	 40	 83	 0.7619048	 0.6066351	 0.67546177
I-LOC	 196	 35	 43	 0.8484849	 0.8200837	 0.83404255
I-PER	 824	 48	 15	 0.94495416	 0.9821216	 0.96317947
B-MISC	 549	 56	 127	 0.90743804	 0.81213015	 0.8571428
B-ORG	 1087	 173	 174	 0.86269844	 0.8620143	 0.8623562
B-PER	 1207	 88	 67	 0.93204635	 0.94740975	 0.9396653
tp: 5888 fp: 647 fn: 735 labels: 8
Macro-average	 prec: 0.8810995, rec: 0.8455454, f1: 0.8629564
Micro-average	 prec: 0.90099466, rec: 0.8890231, f1: 0.8949688
CPU times: user 35.8 ms, sys: 13.2 ms, total: 49 ms
Wall

In [9]:
import pyspark.sql.functions as F
predictions = ner_model.transform(testing_data) 
predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
           .select(F.expr("cols['0']").alias("token"),
                   F.expr("cols['1']").alias("ground_truth"),
                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

22/10/05 20:01:52 WARN TaskSetManager: Stage 5 contains a task of very large size (1011 KiB). The maximum recommended task size is 1000 KiB.
2022-10-05 20:01:53.134675: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice

+--------------+------------+----------+
|token         |ground_truth|prediction|
+--------------+------------+----------+
|CRICKET       |O           |O         |
|-             |O           |O         |
|LEICESTERSHIRE|B-ORG       |B-ORG     |
|TAKE          |O           |O         |
|OVER          |O           |O         |
|AT            |O           |O         |
|TOP           |O           |O         |
|AFTER         |O           |O         |
|INNINGS       |O           |O         |
|VICTORY       |O           |O         |
|.             |O           |O         |
|LONDON        |B-LOC       |B-LOC     |
|1996-08-30    |O           |O         |
|West          |B-MISC      |B-MISC    |
|Indian        |I-MISC      |I-MISC    |
|all-rounder   |O           |O         |
|Phil          |B-PER       |B-PER     |
|Simmons       |I-PER       |I-PER     |
|took          |O           |O         |
|four          |O           |O         |
+--------------+------------+----------+
only showing top

In [10]:
from sklearn.metrics import classification_report
 
preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()
 
print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

22/10/05 20:01:53 WARN TaskSetManager: Stage 6 contains a task of very large size (1011 KiB). The maximum recommended task size is 1000 KiB.


              precision    recall  f1-score   support

       B-LOC       0.91      0.95      0.93      1837
      B-MISC       0.94      0.82      0.87       922
       B-ORG       0.87      0.87      0.87      1341
       B-PER       0.94      0.96      0.95      1842
       I-LOC       0.82      0.82      0.82       257
      I-MISC       0.84      0.62      0.71       346
       I-ORG       0.88      0.74      0.80       751
       I-PER       0.96      0.97      0.96      1307
           O       0.99      1.00      0.99     42759

    accuracy                           0.98     51362
   macro avg       0.91      0.86      0.88     51362
weighted avg       0.98      0.98      0.98     51362



In [13]:
import shutil

shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

'/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en.zip'